# Tutorial 8 - Solver options

In [Tutorial 7](./tutorial-7-model-options.ipynb) we saw how to change the model options. In this tutorial we will show how to modify the solver options. All models in PyBaMM have a default solver which is typically different depending on whether the discretised model results in a system of algebraic equations, ordinary differential equations (ODEs) or differential algebraic equations (DAEs). 

One of the most common options one might want to change is the solver tolerances. By default all tolerances are set to $10^{-6}$. However, depending on your simulation, you may find you want to tighten the tolerances to obtain a more accurate solution, or you may want to loosen the tolerances to reduce the solve time. It is good practice to conduct a tolerance study, where you simulate the same problem with a tighter tolerances and compare the results. We do not show how to do this here, but we give an example of a mesh resolution study in the [next tutorial](./tutorial-9-changing-the-mesh.ipynb), which is conducted in a similar way.

In [ ]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm

Here we will change the absolute and relative tolerances, as well as the "mode" of the `CasadiSolver`, which is the default solver in PyBaMM. For a list of all the solver options  please consult the [documentation](https://docs.pybamm.org/en/latest/source/api/solvers/index.html).

The `CasadiSolver` can operate in a number of modes, including "safe" (default) and "fast". The main difference between these modes is how events are handled. Safe mode performs step-and-check integration and supports event handling (e.g. you can integrate until you hit a certain voltage), and is the recommended for simulations of a full charge or discharge. Fast mode performs direct integration, ignoring events, and is recommended when simulating a drive cycle or other simulation where no events are triggered.

We will solve the DFN with all the default options in both "safe" and "fast" mode and compare the solutions. For both simulations we'll use $10^{-3}$ for both the absolute and relative tolerance. For demonstration purposes we change the cut-off voltage to 3.6V so we can observe the different behaviour of the two solver modes.

In [ ]:
model = pybamm.lithium_ion.DFN()
param = model.default_parameter_values
param["Lower voltage cut-off [V]"] = 3.6

Next we define two instances of the solver, one using the "safe" mode and the other using the "fast" mode. Note how we also pass the tolerances as keyword arguments.

In [ ]:
safe_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="safe")
fast_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="fast")

Then we can create two different simulations (one for each model, where this is set using the `solver` keyword argument) and we solve them. We then plot the results and print the solve time for each simulation.

In [ ]:
# create simulations
safe_sim = pybamm.Simulation(model, parameter_values=param, solver=safe_solver)
fast_sim = pybamm.Simulation(model, parameter_values=param, solver=fast_solver)

# solve
safe_sim.solve([0, 3600])
print(f"Safe mode solve time: {safe_sim.solution.solve_time}")
fast_sim.solve([0, 3600])
print(f"Fast mode solve time: {fast_sim.solution.solve_time}")

# plot solutions
pybamm.dynamic_plot([safe_sim, fast_sim])

We see that both solvers give the same solution and that the "fast" solver, as the name suggests, runs faster. However, if the simulation time was longer the "fast" solver would not notice that the battery is discharging beyond its cut-off voltage and the solver would crash.

Usually the default solver options provide a good combination of speed and accuracy, but we encourage you to investigate different solvers and options to find the best combination for your problem.

In the [next tutorial](./tutorial-9-changing-the-mesh.ipynb) we show how to change the mesh.

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()